<font size="10" color="purple"><left><b><i><u>Exploratory Data Analysis #2</u></i></b></left></font>

<font size="4"><li><left><i>Instanciamos clase EDA con los métodos necesarios</i></left></font>


In [36]:
import pandas as pd

class EDA:
    def read_csv(self, file_path):
        """
        Lee un archivo CSV y devuelve un dataframe.

        Args:
            file_path (str): Ruta al archivo CSV a leer.

        Returns:
            pd.DataFrame: El dataframe con los datos del archivo CSV.
        """
        dataframe = pd.read_csv(file_path,encoding='UTF-8')
        return dataframe

    def get_columnas(self, dataframe):
        """
        Obtiene una lista de las columnas de un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe del cual se desean obtener las columnas.

        Returns:
            list: Lista de las columnas del dataframe.
        """
        columnas = dataframe.columns.tolist()
        return columnas

    def set_columns_dtype_category(self, dataframe, columnas):
        """
        Asigna el tipo de dato "category" a una lista de columnas en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "category".
        """
        dataframe[columnas] = dataframe[columnas].astype('category')

    def set_columns_dtype_int(self, dataframe, columnas):
        """
        Asigna el tipo de dato "int" a una lista de columnas en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "int".
        """
        dataframe[columnas] = dataframe[columnas].astype(int)

    def set_columns_dtype_float(self, dataframe, columnas):
        """
        Asigna el tipo de dato "float" a una lista de columnas en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "float".
        """
        dataframe[columnas] = dataframe[columnas].astype(float)

    def set_columns_dtype_datetime(self, dataframe, columnas, formato='%Y-%m-%d'):
        """
        Asigna el tipo de dato "datetime" a una lista de columnas en un dataframe, con el formato especificado.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "datetime".
            formato (str): El formato de fecha a utilizar (por defecto: "%Y-%m-%d").
        """
        dataframe[columnas] = pd.to_datetime(dataframe[columnas], format=formato)

    def rename_columns(self, dataframe, column_mapping):
        """
        Modifica los nombres de las columnas de un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se le van a modificar los nombres de las columnas.
            column_mapping (dict): Un diccionario que contiene el mapeo de los nombres actuales de las columnas a los nuevos nombres.

        Returns:
            pd.DataFrame: El dataframe con los nombres de columnas modificados.
        """
        dataframe.rename(columns=column_mapping, inplace=True)
        return dataframe

    def get_duplicate_rows(self, dataframe, column):
        """
        Devuelve las filas del dataframe que tienen valores repetidos en una columna específica.

        Args:
            dataframe (pd.DataFrame): El dataframe para buscar filas duplicadas.
            column (str): El nombre de la columna para comprobar duplicados.

        Returns:
            pd.DataFrame: Un dataframe con las filas que tienen valores duplicados en la columna especificada.
        """
        duplicate_rows = dataframe[dataframe.duplicated(subset=[column])]
        return duplicate_rows

    def strip_column_values(self, dataframe, column_names):
        """
        Elimina los espacios en blanco al inicio y al final de los valores de las filas de las columnas especificadas.

        Args:
            dataframe (pd.DataFrame): El dataframe en el cual se van a limpiar los valores de las columnas.
            column_names (list): Una lista de nombres de columnas a las cuales se les va a aplicar el método .strip().

        Returns:
            pd.DataFrame: El dataframe con los valores de las filas de las columnas especificadas limpios de espacios en blanco.
        """
        for column in column_names:
            dataframe[column] = dataframe[column].str.strip()
        return dataframe

    def sort_dataframe(self, dataframe, column, order='a'):
        """
        Ordena un dataframe por una columna especificada.

        Args:
            dataframe (pd.DataFrame): El dataframe a ordenar.
            column (str): El nombre de la columna por la cual ordenar el dataframe.
            order (str, opcional): El orden de clasificación. 'a' para ascendente (predeterminado),
                'd' para descendente.

        Returns:
            pd.DataFrame: El dataframe ordenado por la columna especificada.
        """
        if order == 'd':
            dataframe = dataframe.sort_values(column, ascending=False)
        else:
            dataframe = dataframe.sort_values(column)

        return dataframe

    def sort_and_replace(self, dataframe, column1, column2):
        """
        Ordena un dataframe de menor a mayor según los valores de una columna y reemplaza los valores de otra columna por enteros.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            column1 (str): El nombre de la primera columna según la cual se va a ordenar el dataframe.
            column2 (str): El nombre de la segunda columna cuyos valores se reemplazarán por enteros.

        Returns:
            pd.DataFrame: El dataframe ordenado y con los valores de la segunda columna reemplazados por enteros.
        """
        # Ordenar dataframe por la columna 1 de menor a mayor
        dataframe = dataframe.sort_values(by=column1)

        # Reemplazar los valores de la columna 2 por enteros
        dataframe[column2] = range(len(dataframe))

        return dataframe
        
    def convert_date_format(self, dataframe, columns):
        """
        Convierte el formato de fecha de columnas en un dataframe de 'MMM DD, YYYY' a 'YYYY-MM-DD'
        y asigna el dtype datetime a las columnas especificadas.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            columns (list): Una lista de nombres de columnas a convertir y asignar el dtype datetime.

        Returns:
            pd.DataFrame: El dataframe con los formatos de fecha modificados y dtype datetime asignado.
        """
        for column in columns:
            dataframe[column] = pd.to_datetime(dataframe[column], format='%b %d, %Y').dt.strftime('%Y-%m-%d')
            dataframe[column] = pd.to_datetime(dataframe[column])
            
        return dataframe

    def remove_prefix(self, dataframe, column, prefix):
        """
        Elimina un prefijo de las filas de una columna en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            column (str): El nombre de la columna en la que se eliminará el prefijo.
            prefix (str): El prefijo a eliminar de las filas de la columna.

        Returns:
            pd.DataFrame: El dataframe con el prefijo eliminado de las filas de la columna especificada.
        """
        dataframe[column] = dataframe[column].str.lstrip(prefix)
        return dataframe

# Instancia de la clase EDA
eda = EDA()

<font size="4"><li><left><i>Exportamos Coursera_reviews.csv a dataframe</i></left></font>

In [15]:
# Path de archivos CSV
path = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/e_MOOCs/2_Repo/MOOCs_BA/rawDataSets/Coursera_reviews.csv'

# Leer y almacenar los archivos CSV en dataframes
Coursera_reviews = eda.read_csv(path)


<font size="4"><li><left><i>Obtenemos información del dataframe</i></left></font>

In [16]:
Coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   reviews       1454571 non-null  object
 1   reviewers     1454711 non-null  object
 2   date_reviews  1454711 non-null  object
 3   rating        1454711 non-null  int64 
 4   course_id     1454711 non-null  object
dtypes: int64(1), object(4)
memory usage: 55.5+ MB


<font size="4"><li><left><i>No se identifican valores null en este dataset.</i></left></font><br>

<font size="4"><li><left><i>Clasificamos variables y renombramos columnas:</i></left></font>

In [17]:
# Obtener cabecera del dataframe
Coursera_reviews.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4,google-cbrs-cpi-training


In [39]:
# Obtener lista de columnas
columnas = eda.get_columnas(Coursera_reviews)
print(columnas)

['resenia', 'nombre_revisor', 'fecha_resenia', 'calificacion', 'id_curso']


<font size="4"><li><left><i>Asignamos tipos de datos:</i></left></font>

In [19]:

# Asignar tipo de dato 'category'
# columnas_cat = []
# Coursera_reviews = eda.set_columns_dtype_category(Coursera_reviews,columnas_cat)

# Asignar tipo de dato 'int' a las columnas
# columnas_int = []
# eda.set_columns_dtype_int(dataframe, columnas_int)

# Asignar tipo de dato 'float' a las columnas
# columnas_float = []
# eda.set_columns_dtype_float(dataframe, columnas_float)

# Asignar tipo de dato 'datetime' a la columna con formato 'YYYY-MM-DD'
# columna_fecha = []
# eda.set_columns_dtype_datetime(dataframe, columna_fecha, formato='%Y-%m-%d')

# Crear una lista con las columnas que objetos con formato daytime, asiganar en formato 'MMM DD, YYYY'
columnas_fecha = ['date_reviews']
Coursera_reviews = eda.convert_date_format(Coursera_reviews, columnas_fecha)


In [22]:
# Obtener cabecera del dataframe
Coursera_reviews.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,2020-02-12,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,2020-09-28,4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,2020-04-08,4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,2020-02-24,4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,2020-06-18,4,google-cbrs-cpi-training


In [20]:
Coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   reviews       1454571 non-null  object        
 1   reviewers     1454711 non-null  object        
 2   date_reviews  1454711 non-null  datetime64[ns]
 3   rating        1454711 non-null  int64         
 4   course_id     1454711 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 55.5+ MB


<font size="4"><li><left><i>Renombramos columnas:</i></left></font>

In [24]:
# Definir un diccionario de mapeo de nombres de columnas
column_mapping = {'reviews':'resenia', 'reviewers': 'nombre_revisor', 'date_reviews':'fecha_resenia', 'rating':'calificacion', 'course_id':'id_curso'}

# Llamar al método rename_columns
dataframe_modificado = eda.rename_columns(Coursera_reviews, column_mapping)


In [25]:
Coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   resenia         1454571 non-null  object        
 1   nombre_revisor  1454711 non-null  object        
 2   fecha_resenia   1454711 non-null  datetime64[ns]
 3   calificacion    1454711 non-null  int64         
 4   id_curso        1454711 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 55.5+ MB


<font size="4" color="white"><li><left><i>Eliminamos espacios antes/despues:</i></left></font>

In [27]:
# Eliminar espacios alrededor de str
# Definir una lista de nombres de columnas a limpiar
columnas_a_limpiar = ['resenia', 'nombre_revisor','id_curso']

# Llamar al método strip_column_values
Coursera_courses = eda.strip_column_values(Coursera_reviews, columnas_a_limpiar)

<font size="4" color="white"><li><left><i>Eliminamos el prefijo `By `de los valores de la columna `nombre_revisor`:</i></left></font>

In [30]:
Coursera_courses = eda.remove_prefix(Coursera_courses, 'nombre_revisor', 'By ')
Coursera_courses.head()

,resenia,nombre_revisor,fecha_resenia,calificacion,id_curso
0,"Pretty dry, but I was able to pass with just t...",Robert S,2020-02-12,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,Gabriel E R,2020-09-28,4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,Jacob D,2020-04-08,4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,Dale B,2020-02-24,4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,Sean G,2020-06-18,4,google-cbrs-cpi-training


<font size="4" color="white"><li><left><i>Ordenamos por columna:</i></left></font>

In [33]:
# Llamar al método sort_dataframe
courcera_courses = eda.sort_dataframe(Coursera_courses, 'calificacion','d')
courcera_courses.head()

,resenia,nombre_revisor,fecha_resenia,calificacion,id_curso
1454710,G,Reza S,2018-08-12,5,computer-networking
783268,I love it. Learnt a lot from this online course.,Tan Y S,2020-05-12,5,search-engine-optimization
783274,One of my best course that I want to finish it .,Fadli I,2016-02-10,5,search-engine-optimization
783273,Thank you so much for providing me this platform,Simranjit S,2016-10-14,5,search-engine-optimization
783272,A very useful introduction to the basics of SEO.,Alejandro F y B,2019-09-25,5,search-engine-optimization


 <font size="4" color="white"><li><left><i>Analizamos duplicados:</i></left></font>

In [47]:
# Contamos valores repetidos en 'resenia'
value_counts = Coursera_courses['resenia'].value_counts()

# Calculamos el porcentaje de cada línea agrupada
percentage = value_counts / len(Coursera_courses) * 100

# Creamos un dataframe auxiliar con las columnas 'calificacion', 'cantidad' y 'porcentaje'
aux_df = pd.DataFrame({'cantidad': value_counts, 'porcentaje': percentage})

# Mostramos el dataframe auxiliar
print(aux_df)

                                                    cantidad  porcentaje
good                                                    9236    0.634903
Good                                                    6106    0.419740
Excellent                                               4656    0.320064
Great course!                                           2796    0.192203
Excellent course                                        2409    0.165600
...                                                      ...         ...
Great course. I learned a lot with this and rea...         1    0.000069
Great class and I learned a lot! thank you for ...         1    0.000069
Fantastic ! Now I am in the information era! Th...         1    0.000069
Detailed explanation in a simple way helps to g...         1    0.000069
Pretty dry, but I was able to pass with just tw...         1    0.000069

[454690 rows x 2 columns]


<font size="4"><li><left><i>`good` es la reseña que más se repite.</i></left></font>

In [38]:
# Traer filas con duplicados
filas_duplicadas = eda.get_duplicate_rows(Coursera_courses, 'resenia')
filas_duplicadas.head(20)

,resenia,nombre_revisor,fecha_resenia,calificacion,id_curso
29,Great course,Landon S,2020-02-09,5,google-cbrs-cpi-training
30,Great course,Mark B,2019-12-03,5,google-cbrs-cpi-training
31,Solid presentation all the way through. I real...,Logan D,2020-09-03,5,google-cbrs-cpi-training
32,Probably the best certification course I've ta...,Luis M C,2019-11-21,5,google-cbrs-cpi-training
33,The ProctorU.com system took 2 times the amoun...,scott w,2020-09-28,5,google-cbrs-cpi-training
34,Covered all of the required information in an ...,Ryan H,2019-08-26,5,google-cbrs-cpi-training
35,"Great course, lectures were straight forward a...",Samuel D,2020-01-24,5,google-cbrs-cpi-training
36,The course was straight forward and prepared m...,Scot A W,2020-05-01,5,google-cbrs-cpi-training
37,Pretty well designed course. Few things on the...,Jonathan L,2020-01-17,5,google-cbrs-cpi-training
38,"Great course, i have feedback about one questi...",Geoff T,2019-07-21,5,google-cbrs-cpi-training


<font size="4"><li><left><i>Se define que el curso no es necesariamente el mismo, se mantienen ambas filas.</i></left></font>

In [45]:
# Contamos valores repetidos en 'nombre_revisor'
value_counts = Coursera_courses['nombre_revisor'].value_counts()

# Calculamos el porcentaje de cada línea agrupada
percentage = value_counts / len(Coursera_courses) * 100

# Creamos un dataframe auxiliar con las columnas 'calificacion', 'cantidad' y 'porcentaje'
aux_df = pd.DataFrame({'cantidad': value_counts, 'porcentaje': percentage})

# Mostramos el dataframe auxiliar
print(aux_df)

             cantidad  porcentaje
Deleted A        5412    0.372033
Muhammad A        664    0.045645
Abhishek S        440    0.030247
David M           439    0.030178
Michael S         431    0.029628
...               ...         ...
Chinta S K          1    0.000069
Lucas A M S         1    0.000069
Facundo J           1    0.000069
Tarun d             1    0.000069
Andrij S            1    0.000069

[283549 rows x 2 columns]


<font size="4"><li><left><i>`Deleted A` es el `nombre_revisor` que más se repite y representa un `0,37%` del total.  </i></left></font>

In [46]:
# Contamos valores repetidos en 'calificacion'
value_counts = Coursera_courses['calificacion'].value_counts()

# Calculamos el porcentaje de cada línea agrupada
percentage = value_counts / len(Coursera_courses) * 100

# Creamos un dataframe auxiliar con las columnas 'calificacion', 'cantidad' y 'porcentaje'
aux_df = pd.DataFrame({'cantidad': value_counts, 'porcentaje': percentage})

# Mostramos el dataframe auxiliar
print(aux_df)


   cantidad  porcentaje
5   1146164   78.789808
4    226702   15.583989
3     48303    3.320453
1     17354    1.192952
2     16188    1.112798


<font size="4"><li><left><i>Todos valores únicos para `url_curso`.</i></left></font>

In [44]:
# Contamos valor repetidos en 'id_curso'
Coursera_courses['id_curso'].value_counts()

machine-learning                             1
english-composition                          1
wharton-communication-skills                 1
introduction-trading-machine-learning-gcp    1
python-programming-introduction              1
                                            ..
solar-energy-basics                          1
bootstrap-4                                  1
google-cloud-java-spring                     1
forensic-science                             1
data-science-course                          1
Name: id_curso, Length: 623, dtype: int64

<font size="4"><li><left><i>Todos valores únicos para `id_curso`. Se reemplar valores numericos por</i></left></font>

In [50]:
Coursera_courses = eda.sort_and_replace(Coursera_courses, 'nombre_curso', 'id_curso')


In [51]:
Coursera_courses.head()

,nombre_curso,institucion,url_curso,id_curso
261,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,https://www.coursera.org/learn/crash-course-in...,0
622,A Crash Course in Data Science,Johns Hopkins University,https://www.coursera.org/learn/data-science-co...,1
396,A Law Student's Toolkit,Yale University,https://www.coursera.org/learn/law-student,2
9,AI For Everyone,DeepLearning.AI,https://www.coursera.org/learn/ai-for-everyone,3
215,AI for Medical Diagnosis,DeepLearning.AI,https://www.coursera.org/learn/ai-for-medical-...,4


In [52]:
Coursera_courses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 261 to 365
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nombre_curso  623 non-null    object
 1   institucion   623 non-null    object
 2   url_curso     623 non-null    object
 3   id_curso      623 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 24.3+ KB
